In [7]:
import numpy as np
import pandas as pd
! pip install dowhy
import dowhy
from dowhy import CausalModel

In [8]:
# 创建一个示例数据集
np.random.seed(42)
n_days = 100
data = pd.DataFrame({
    'date': pd.date_range(start='2024-01-01', periods=100),
    'marketing_campaign': ['A', 'B', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A'] + ['C'] * 90,
    'weather': ['cloudy', 'rainy', 'rainy', 'rainy', 'rainy'] + ['sunny'] * 95,
    'visitors': [1795, 2051, 2643, 4627, 2363] + [3799, 3191, 1608, 4272, 2147] * 19,
    'conversion_rate': [0.035246, 0.040874, 0.045751, 0.029779, 0.026607] + [0.030759, 0.027625, 0.028859, 0.029677, 0.045460] * 19
})

# 增加一个人为引入的因果关系 - 假设某个营销活动在第50天开始
data.loc[data['date'] >= '2024-02-20', 'marketing_campaign'] = 'A'
data


,date,marketing_campaign,weather,visitors,conversion_rate
0,2024-01-01,A,cloudy,1795,0.035246
1,2024-01-02,B,rainy,2051,0.040874
2,2024-01-03,A,rainy,2643,0.045751
3,2024-01-04,A,rainy,4627,0.029779
4,2024-01-05,A,rainy,2363,0.026607
...,...,...,...,...,...
95,2024-04-05,A,sunny,3799,0.030759
96,2024-04-06,A,sunny,3191,0.027625
97,2024-04-07,A,sunny,1608,0.028859
98,2024-04-08,A,sunny,4272,0.029677


In [10]:
unique_campaigns = data['marketing_campaign'].unique()
print("Unique campaigns:", unique_campaigns)

Unique campaigns: ['A' 'B' 'C']


In [12]:
# 定义因果模型
model = CausalModel(
    data=data,
    treatment='marketing_campaign',
    outcome='visitors',
    common_causes=['weather']
)

# 识别因果关系
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)

# 估计因果效应
estimate = model.estimate_effect(
    identified_estimand,
    method_name="backdoor.linear_regression",
    control_value=0,
    treatment_value=1
)

# 打印因果效应估计值
print(estimate)

# 进一步检查因果效应
refute_results = model.refute_estimate(
    identified_estimand,
    estimate,
    method_name="random_common_cause"
)

# 打印因果效应检验结果
print(refute_results)

C:\Users\Steven\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\Steven\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


ValueError: Found unknown categories [1] in column 0 during transform